In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
class TSCANToEnd(keras.Model):
    
    def __init__(self, model):
        super().__init__()
        self.inner = model
    
    def call(self, x, training=None):
        x_ = x[:, 1:] - x[:, :-1]
        x_ = (x_ - ops.mean(x_, axis=(2,3 ), keepdims=True))/(ops.std(x_, axis=(2, 3), keepdims=True)+1e-6)
        return self.inner((ops.concatenate([x_, x_[:, -1:]], axis=1), ops.mean(x, axis=(1, ), keepdims=True)), training=training)

model = TSCANToEnd(TSCAN())
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 36, 36, 3)));
model.inner.summary()

Model: "tscan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (640, 34, 34, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (640, 15, 15, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 34, 34, 32)        │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 15, 15, 64)        │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 34, 34, 1)         │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 15, 15, 1)         │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (640, 17, 17, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (640, 1)               │       401,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,899 (2.03 MB)

 Trainable params: 532,899 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
@jax.jit
@partial(shard_map, mesh=mesh, in_specs=P('x'), out_specs=P('x'))
def diff_label(x, y):
    y = ops.concatenate([y[:,1:]-y[:,:-1], ops.zeros_like(y[:,:1])], axis=1)
    return x, y

batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x36x36_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)
training_set = training_set.apply_fn(diff_label)
validation_set = validation_set.apply_fn(diff_label)

rlap_160x36x36_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x36x36_all             datatape has been loaded.     2944 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tscan.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/tscan.weights.h5')
model.load_weights('../weights/tscan.weights.h5')

Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.1713	Validation loss:0.1467	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1455	Validation loss:0.1335	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1452	Validation loss: 0.133	


In [5]:
model = TSCANToEnd(TSCAN())
model(np.random.random((4, 160, 36, 36, 3)));
model.load_weights('../weights/tscan.weights.h5')

In [6]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_RLAP_MMPD.h5', cumsum=True)
get_metrics('../results/TSCAN_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,14.341±0.194,21.877±3.273,16.626±0.224,0.27306
Whole,10.192±0.439,15.167±4.33,11.62±0.47,0.47127


In [7]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_RLAP_COHFACE.h5', cumsum=True, fps=30)
get_metrics('../results/TSCAN_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,9.372±0.314,16.354±3.969,13.949±0.502,0.34388
Whole,6.58±0.558,9.714±4.034,10.114±0.987,0.6098


In [8]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_RLAP_PURE.h5', cumsum=True)
get_metrics('../results/TSCAN_RLAP_PURE.h5') 

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,4.794±0.44,13.052±4.948,7.601±0.74,0.83995
Whole,3.902±0.764,7.045±3.817,6.215±1.23,0.95917


In [9]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_RLAP_UBFC.h5', cumsum=True)
get_metrics('../results/TSCAN_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.822±0.166,4.209±2.323,1.881±0.166,0.96992
Whole,0.904±0.214,1.658±1.161,0.902±0.207,0.99569
